In [76]:
import os
from google.cloud import storage
from google.cloud import aiplatform
from vertexai.generative_models import GenerativeModel, Part
import PyPDF2
import json
import re

In [67]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/dominiclakshan/Desktop/jupyter_notebooks/pdf_splitter/ui-ux-model-6e8765c9a1a6.json"

In [68]:
aiplatform.init(project="ui-ux-model", location="us-central1")

In [69]:
storage_client = storage.Client()

In [70]:
model_name = "gemini-1.5-flash-preview-0514"

In [71]:
model = GenerativeModel(model_name=model_name)

In [72]:
bucket_name = 'test-pdfs-store'
input_folder = 'input_files'
output_folder = 'output_files'

In [73]:
bucket = storage_client.get_bucket(bucket_name)

In [74]:
blobs = list(bucket.list_blobs(prefix=f"{input_folder}/"))


In [75]:
pdf_files = [blob.name.split('/')[-1] for blob in blobs if blob.name.endswith('.pdf')]

In [79]:
for pdf_file in pdf_files:
    pdf_file_uri= f"gs://{bucket_name}/{input_folder}/{pdf_file}"
    prompt = """
    Analyze the PDF and provide the page breaks for each chapter.
    Return the result as a JSON object with the following format:
    {
      "page_breaks": [5, 10, 15]
    }
    """
    
    pdf_file = Part.from_uri(pdf_file_uri, mime_type="application/pdf")
    contents = [pdf_file, prompt]
    response = model.generate_content(contents)
    print(response)
    response_str = str(response)
    
    # Check if the response is not empty before processing
    if response_str:
        try:
            # Use regular expressions to extract the page breaks
            page_breaks_pattern = r'"text":\s*"(.*?)"'
            match = re.search(page_breaks_pattern, response_str)
            if match:
                page_breaks_str = match.group(1)
                page_breaks = [int(x) for x in page_breaks_str.split(",")]
                print("Page Breaks:", page_breaks)
            else:
                print("Page breaks not found in the response.")
        except Exception as e:
            print("Error processing the response:", e)
    else:
        print("Empty response received.")


candidates {
  content {
    role: "model"
    parts {
      text: "```json\n{\n  \"page_breaks\": [\n    5,\n    10,\n    12\n  ]\n}\n```"
    }
  }
  finish_reason: STOP
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
    probability_score: 0.11240508407354355
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.08314290642738342
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: NEGLIGIBLE
    probability_score: 0.10781263560056686
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.04443143308162689
  }
  safety_ratings {
    category: HARM_CATEGORY_HARASSMENT
    probability: NEGLIGIBLE
    probability_score: 0.13534632325172424
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.09467770159244537
  }
  safety_ratings {
    category: HARM_CATEGORY_SEXUALLY_EXPLICIT
    probability: NEGLIGIBLE
    probability_score: 0.14175598323345184
    severity: HARM_SEVERITY_NEGLIGIBLE
    sever